In [2]:
import warnings
warnings.filterwarnings('ignore') # Filter out warnings
import numpy as np
import pandas as pd
import random as rnd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
from scipy import stats
import math
from math import isnan

In [3]:
# Create a Combined Dataframe from message csv and orderbook csv
def Merge_MsgAndOrdb(msg_path,ordb_path):
    # get the level of the LOBSTER dataset
    level = int(msg_path.split(".")[0][-1])
    # get the date 
    date = msg_path.split(".")[0].split("_")[1]
    
    # add header row for message csv
    msg_names = ["Time_stamp", "Type", "OrderID", "Size","Price","Direction"]
    df_msg = pd.read_csv(msg_path,names=msg_names)
    # add header row for orderbook csv
    default_ordb_names = ['Ask Price','Ask Size','Bid Price','Bid Size']
    ordb_names = []
    for i in range(level):
        for item in default_ordb_names:
            ordb_names.append(str(item)+str(" ")+str(i+1))
    df_ordb = pd.read_csv(ordb_path,names=ordb_names)
    
    # adding a meaningful time column
    df_msg['Time'] = pd.to_datetime(df_msg['Time_stamp'],unit="s",origin=pd.Timestamp(date))
    
    # combine two dataframes and return it
    df = pd.concat([df_msg,df_ordb],axis=1)
    return df

In [4]:
df = Merge_MsgAndOrdb(msg_path='AMZN_2012-06-21_34200000_57600000_message_5.csv',
                     ordb_path='AMZN_2012-06-21_34200000_57600000_orderbook_5.csv')
df.head(5)

,Time_stamp,Type,OrderID,Size,Price,Direction,Time,Ask Price 1,Ask Size 1,Bid Price 1,...,Bid Price 3,Bid Size 3,Ask Price 4,Ask Size 4,Bid Price 4,Bid Size 4,Ask Price 5,Ask Size 5,Bid Price 5,Bid Size 5
0,34200.017460,5,0,1,2238200,-1,2012-06-21 09:30:00.017460,2239500,100,2231800,...,2230400,100,2242500,100,2230000,10,2244000,547,2226200,100
1,34200.189608,1,11885113,21,2238100,1,2012-06-21 09:30:00.189608,2239500,100,2238100,...,2230700,200,2242500,100,2230400,100,2244000,547,2230000,10
2,34200.189608,1,3911376,20,2239600,-1,2012-06-21 09:30:00.189608,2239500,100,2238100,...,2230700,200,2240000,220,2230400,100,2242500,100,2230000,10
3,34200.189608,1,11534792,100,2237500,1,2012-06-21 09:30:00.189608,2239500,100,2238100,...,2231800,100,2240000,220,2230700,200,2242500,100,2230400,100
4,34200.189608,1,1365373,13,2240000,-1,2012-06-21 09:30:00.189608,2239500,100,2238100,...,2231800,100,2240000,233,2230700,200,2242500,100,2230400,100


In [5]:
cancelledOrderID = set(df[(df['Type']==2)].OrderID.tolist())
#len(cancelledOrderID)
lookup = {1:'Submission', 2:'Cancellation', 3:'Deletion', 4:'Execution Visible',5:'Execution Hidden',7:'Trading Halt'}

In [6]:
def RenameColumn(df,state_type):
    old_col = df.columns.tolist()
    new_col = []
    for col in old_col:
        if ( ("Bid" in col) | ("Ask" in col) ):
            new_col.append(col+str(" ")+state_type)
        elif ((col == 'Size') | (col == 'Price')):
            new_col.append(col+str(" ")+state_type)  
        elif (col=='Time'):
            new_col.append(col+str(" ")+state_type)
        else:
            new_col.append(col)
    columns = dict(zip(old_col,new_col))
    df.rename(columns=columns,inplace=True)
    return 

In [7]:
def GroupByOrderType(df,typeID):                        # for now, typeID should only be 2 or 3 or 4 or 5
    # lookup dictionary
    lookup = {1:'Submission', 2:'Cancellation', 3:'Deletion', 4:'Execution Visible',5:'Execution Hidden',7:'Trading Halt'}
    # getting all the orders of type typeID
    orderID_set = set(df[(df['Type']==typeID)].OrderID.tolist())
    df_subset = df.loc[df['OrderID'].isin(orderID_set)]
    
    # get initial states when orders of this type is submitted
    df_init = df_subset[(df_subset['Type']==1)]
    RenameColumn(df_init,state_type=lookup[1])
    
    # get the end states of type typeID
    df_end = df_subset[(df_subset['Type']==typeID)]
    RenameColumn(df_end,state_type=lookup[typeID])
    
    # merge these two dataframes
    df = df_init.merge(df_end, left_on='OrderID', right_on='OrderID', how='outer')

    return df

In [8]:
df_OC = GroupByOrderType(df,typeID=2)
df_Executed = GroupByOrderType(df,typeID=4)
print(len(df_OC.index))
print(len(df_Executed.index))

450
8974


In [9]:
print(len(df[(df['Type']==2)].index))
print(len(df[(df['Type']==4)].index))

450
8974


In [10]:
#df_OC.columns

In [11]:
#df_OC[["Time_stamp_x","Time_stamp_y"]].head(10)

In [12]:
def AddLifetime(df):
    df_new = df.copy(deep=False)
    lifetime = []
    for index, row in df_new.iterrows():
        # check if the keys exist
        try:
            dummy = row['Time_stamp_y'] - row['Time_stamp_x']
        except KeyError:
            print("columns not exist.")
        lifetime.append(dummy)
    df_new['lifetime'] = lifetime
    df_new = df_new.dropna(how='any')
    return df_new

In [13]:
df_OC = AddLifetime(df_OC)
df_Executed = AddLifetime(df_Executed)
#df_OC.head(5)

In [14]:
def AddVoltsAndNumberOfSumbittedOrders(df,dt_mins,df_original):
    # make a copy 
    df_new = df.copy(deep=False)
    # obtain a df of executed visible orders and 
    #        a df of submitted orders
    df_submitted = GroupByOrderType(df_original,typeID=1).dropna(how='any')
    df_visExecute = GroupByOrderType(df_original,typeID=4).dropna(how='any')
    # convert delta t to seconds
    dt = dt_mins * 60
    # get submission times of all orders in the df
    try:
        submission_times = df_new[['OrderID','Time_stamp_x']]
    except KeyError:
        print("columns not exist.")    
    # now, gather all executed order prices between
    # the above times and (times-dt)
    norders,volts = [],[]
    # iterate through each row
    for index, row in submission_times.iterrows():
        t = row['Time_stamp_x']
        submitted_order = df_submitted[(df_submitted["Time_stamp_x"] > t-dt) & (df_submitted["Time_stamp_x"] < t)]
        executed_order = df_visExecute[(df_visExecute["Time_stamp_y"] > t-dt) & (df_visExecute["Time_stamp_y"] < t)]
        # computing...
        norder = len(submitted_order['OrderID'])
        volt = np.std(executed_order['Price Submission'])
        # appending to corresponding lists
        norders.append(norder)
        volts.append(volt)
    # appending the results 
    df_new['Volatility'] = volts
    df_new['Number of Orders Submitted'] = norders
    return df_new

In [15]:
df_OC = AddVoltsAndNumberOfSumbittedOrders(df_OC,dt_mins=15,df_original=df)
df_Executed = AddVoltsAndNumberOfSumbittedOrders(df_Executed,dt_mins=15,df_original=df)
#df_Executed.tail(5)

In [16]:
def AddSpreadTwoStates(df):
    # make a copy
    df_new = df.copy(deep=False)
    # lookup dictionary
    lookup = {1:'Submission', 2:'Cancellation', 3:'Deletion', 4:'Execution Visible',5:'Execution Hidden',7:'Trading Halt'}
    # gather the order type
    for col in df_new.columns.tolist():
        if 'Cancellation' in col:
            orderType = 'Cancellation'
            break
        elif 'Execution Visible' in col:
            orderType = 'Execution Visible'
            break
    
    # define designated columns at end state
    ask_end = 'Ask Price 1 ' + orderType
    bid_end = 'Bid Price 1 ' + orderType
    # adding spreads at start and end states
    spreads_start,spreads_end = [],[]
    for index, row in df_new.iterrows():
        try:
            spread_start = row['Ask Price 1 Submission'] - row['Bid Price 1 Submission']
            spread_end = row[ask_end] - row[bid_end]
        except KeyError:
            print('Columns not exist.')
        spreads_start.append(spread_start)
        spreads_end.append(spread_end)
    # append them to the dataframe
    df_new['Effective Spread Start'] = spreads_start
    df_new['Effective Spread End'] = spreads_end
    # return
    return df_new

In [17]:
df_OC = AddSpreadTwoStates(df_OC)
df_Executed = AddSpreadTwoStates(df_Executed)

In [18]:
# of all EXECUTION ORDERS
#df_Executed.head(5)

In [19]:
# of all CANCELLED ORDERS
#df_OC.head(5)

## Stuff below: referenced from paper 462

In [20]:
print(df.columns.values)

['Time_stamp' 'Type' 'OrderID' 'Size' 'Price' 'Direction' 'Time'
 'Ask Price 1' 'Ask Size 1' 'Bid Price 1' 'Bid Size 1' 'Ask Price 2'
 'Ask Size 2' 'Bid Price 2' 'Bid Size 2' 'Ask Price 3' 'Ask Size 3'
 'Bid Price 3' 'Bid Size 3' 'Ask Price 4' 'Ask Size 4' 'Bid Price 4'
 'Bid Size 4' 'Ask Price 5' 'Ask Size 5' 'Bid Price 5' 'Bid Size 5']


In [21]:
def ComputeTimeInsenstiveSet(df_original, levels=5):
    df = df_original.copy(deep=False)
    for i in range(1,levels+1):
        # compute bid-ask spreads and mid-prices (v2)
        df["bid-ask spread {}".format(i)] = df["Ask Price {}".format(i)] - df["Bid Price {}".format(i)]
        df["mid-price {}".format(i)] = (df["Ask Price {}".format(i)] + df["Bid Price {}".format(i)])/2
        # compute price differences (v3)
        df["d_P_ask_{}{}".format(levels,1)] = df["Ask Price {}".format(levels)] - df["Ask Price 1"]
        df["d_P_bid_{}{}".format(levels,1)] = df["Bid Price {}".format(levels)] - df["Bid Price 1"]
        for j in range(1,levels):
            df["d_P_ask_{}{}".format(j+1,j)] = np.abs(df["Ask Price {}".format(j+1)] - df["Ask Price {}".format(j)])
            df["d_P_bid_{}{}".format(j+1,j)] = np.abs(df["Bid Price {}".format(j+1)] - df["Bid Price {}".format(j)])
        # compute mean prices and volumes (v4)
        df["Mean_P_ask"] = df[["Ask Price {}".format(i) for i in range(1,levels+1)]].mean(axis=1)
        df["Mean_P_bid"] = df[["Bid Price {}".format(i) for i in range(1,levels+1)]].mean(axis=1)
        df["Mean_V_ask"] = df[["Ask Size {}".format(i) for i in range(1,levels+1)]].mean(axis=1)
        df["Mean_V_bid"] = df[["Bid Size {}".format(i) for i in range(1,levels+1)]].mean(axis=1)
        # compute accumulated differences (v5)
        df["P_accumulated"] = df[["Ask Price {}".format(i) for i in range(1,levels+1)]].sum(axis=1) \
            - df[["Bid Price {}".format(i) for i in range(1,levels+1)]].sum(axis=1)
        df["V_accumulated"] = df[["Ask Size {}".format(i) for i in range(1,levels+1)]].sum(axis=1) \
            - df[["Bid Size {}".format(i) for i in range(1,levels+1)]].sum(axis=1)
    # returning...
    return df

In [22]:
df = ComputeTimeInsenstiveSet(df)
#df[["Ask Price 1", "Bid Price 1", "bid-ask spread 1", "mid-price 1", "d_P_ask_32", "Mean_P_ask"]].head(10)
#df[["Ask Price 1", "Ask Price 2", "Ask Price 3", "Ask Price 4", "Ask Price 5", "Mean_P_ask"]].head(10)
#df.head(10)
print(df.shape)
print(df.shape[0] * df.shape[1] * 8 / (1024**3))

(155935, 53)
0.06157573312520981


In [60]:
def ComputeTimeSensitiveSet(df_original,levels=5,dt_secs=1): 
    df = df_original.copy(deep=False)
    # 
    for index, row in df.iterrows():
        t = row['Time_stamp']
        temp = df.loc[(df['Time_stamp']>t-dt_secs)&(df['Time_stamp']<=t)]
        for i in range(1,levels+1):
            # compute price and volume derivatives (v6)
            df.set_value(index,'der_P_ask_{}'.format(i),
                         (temp['Ask Price {}'.format(i)].iloc[-1]-temp['Ask Price {}'.format(i)].iloc[0])/dt_secs) 
            df.set_value(index,'der_P_bid_{}'.format(i),
                         (temp['Bid Price {}'.format(i)].iloc[-1]-temp['Bid Price {}'.format(i)].iloc[0])/dt_secs) 
            df.set_value(index,'der_V_ask_{}'.format(i),
                         (temp['Ask Size {}'.format(i)].iloc[-1]-temp['Ask Size {}'.format(i)].iloc[0])/dt_secs) 
            df.set_value(index,'der_V_bid_{}'.format(i),
                         (temp['Bid Size {}'.format(i)].iloc[-1]-temp['Bid Size {}'.format(i)].iloc[0])/dt_secs) 
            # compute average intensity of each type (v7)

    return df

In [61]:
df_test = ComputeTimeSensitiveSet(df_original=df.head(5))

In [62]:
#df_test.columns

Index(['Time_stamp', 'Type', 'OrderID', 'Size', 'Price', 'Direction', 'Time',
       'Ask Price 1', 'Ask Size 1', 'Bid Price 1', 'Bid Size 1', 'Ask Price 2',
       'Ask Size 2', 'Bid Price 2', 'Bid Size 2', 'Ask Price 3', 'Ask Size 3',
       'Bid Price 3', 'Bid Size 3', 'Ask Price 4', 'Ask Size 4', 'Bid Price 4',
       'Bid Size 4', 'Ask Price 5', 'Ask Size 5', 'Bid Price 5', 'Bid Size 5',
       'bid-ask spread 1', 'mid-price 1', 'd_P_ask_51', 'd_P_bid_51',
       'd_P_ask_21', 'd_P_bid_21', 'd_P_ask_32', 'd_P_bid_32', 'd_P_ask_43',
       'd_P_bid_43', 'd_P_ask_54', 'd_P_bid_54', 'Mean_P_ask', 'Mean_P_bid',
       'Mean_V_ask', 'Mean_V_bid', 'P_accumulated', 'V_accumulated',
       'bid-ask spread 2', 'mid-price 2', 'bid-ask spread 3', 'mid-price 3',
       'bid-ask spread 4', 'mid-price 4', 'bid-ask spread 5', 'mid-price 5',
       'der_P_ask_1', 'der_P_bid_1', 'der_V_ask_1', 'der_V_bid_1',
       'der_P_ask_2', 'der_P_bid_2', 'der_V_ask_2', 'der_V_bid_2',
       'der_P_ask_3', '

In [49]:
def AddLabels(df_original, levels=5):  # based on page 16 of paper 462
    df = df_original.copy(deep=False)
    for index, row in df.iterrows():
        df["bid-ask spread label"]

In [24]:
#df.head(5)

## Everything below seems to be your codes of making plots... if you wanna clean them up feel free to do that

In [25]:
### Determine the cancellation rate during dt
plt.hist(df_test["Time_stamp_y"], bins=25)
start = df_test["Time_stamp_y"].min()
end = df_test["Time_stamp_y"].max()
print(start, end)
n = (end-start)/dt
Dt = np.linspace(start,end,n)
for i in Dt:
    plt.axvline(i, ls=":", color="k", alpha=0.5)
plt.title("Visual of approximate cancellation rate discretization")
plt.show()

NameError: name 'df_test' is not defined

In [ ]:
start_of_day = 9.5*3600 # 9:30 AM
end_of_day = 16*3600 # 4 PM
dt = 6.5*3600
n = (end_of_day - start_of_day)/dt
bins = np.linspace(start_of_day, end_of_day, n+1)
#print(dt)
#print(bins)
cancelled = plt.hist(df_test["Time_stamp_y"], bins=bins, alpha=0.5, histtype="step")
submitted = plt.hist(df_submitted["Time_stamp_x"], bins=bins, alpha=0.5, histtype="step")

print("Cancellation Ratio during dt:")
cancellation_ratio_during_dt = cancelled[0]/submitted[0]
print(cancellation_ratio_during_dt)

print("Cancellation Rate:")
cancellation_rate = cancelled[0]/dt
print(cancellation_rate)

In [ ]:
len(set(df[(df['Type']==2)]['OrderID'].tolist()))

In [ ]:
ks = [1,2,3,4,5,7]
for k in ks:
    print(len(set(df[(df['Type']==k)]['OrderID'].tolist())))

In [ ]:
df_test.columns

In [ ]:
data = df_test[["OrderID","Time Submission","Direction_x","Price Submission","Size Submission",
                "Time Cancellation","lifetime","Size Cancellation",
                "Ask Price 1 Submission","Bid Price 1 Submission","Ask Price 1 Cancellation","Bid Price 1 Cancellation"]]
data["Effective Spread Submission"] = data['Ask Price 1 Submission'] - data['Bid Price 1 Submission']
data["Effective Spread Cancellation"] = data['Ask Price 1 Cancellation'] - data['Bid Price 1 Cancellation']
data["Volatility"] = volts
data["Number of Orders Submitted"] = norders
#len(data.index)
data.head(10)

In [ ]:
data.to_csv("AMZN_cancelledOrders.csv")

In [ ]:
## plotting...
x = data['Time Submission']
y = data['Price Submission']
plt.plot(x,y)
plt.xlabel("Time Submission")
plt.ylabel("")

In [ ]:
lookup = {1:'Submission', 2:'Cancellation', 3:'Deletion', 4:'Execution Visible',5:'Execution Hidden',7:'Trading Halt'}
for values in lookup.values():
    print(values)

In [ ]:
df1 = pd.DataFrame({'a':range(1,5), 'b':['a','b','c','d']})
df1

In [ ]:
df1["a"].iloc[-1] 